<a href="https://colab.research.google.com/github/Ayesha0300/Projects/blob/main/openrouter_ai_agents_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [4]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [17]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "moonshotai/kimi-vl-a3b-thinking:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [18]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745742788-7KxT4FSmmkU1eppoHe62', 'provider': 'Chutes', 'model': 'moonshotai/kimi-vl-a3b-thinking:free', 'object': 'chat.completion', 'created': 1745742788, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': '◁think▷Alright, the user asked, "What is the meaning of life?" That\'s a big question. I need to break it down. First, I should acknowledge that it\'s a complex and personal topic. People have different philosophical or religious views on this.\n\nI should probably mention some common perspectives. Maybe introduce the existentialist idea that life has no inherent meaning, but individuals create their own meaning. Then talk about religious or spiritual viewpoints, like in religions such as Christianity or Buddhism, where life might have a purpose related to a higher power or spiritual growth.\n\nDon\'t forget secular perspectives. Some people might approach it through science or

In [19]:
data = response.json()
data['choices'][0]['message']['content']

'◁think▷Alright, the user asked, "What is the meaning of life?" That\'s a big question. I need to break it down. First, I should acknowledge that it\'s a complex and personal topic. People have different philosophical or religious views on this.\n\nI should probably mention some common perspectives. Maybe introduce the existentialist idea that life has no inherent meaning, but individuals create their own meaning. Then talk about religious or spiritual viewpoints, like in religions such as Christianity or Buddhism, where life might have a purpose related to a higher power or spiritual growth.\n\nDon\'t forget secular perspectives. Some people might approach it through science or personal fulfillment. Emphasize that there\'s no one correct answer, which is okay, as it\'s a question many explore throughout their lives.\n\nAlso, consider cultural differences in midlife crises and how different societies address the concept of life\'s meaning. Maybe mention historical figures who\'ve ponde

## 2. Using OpenAI Agents SDK

In [22]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in English.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in business programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

◁think▷Okay, so I need to explain recursion in business programming. Let me start by recalling what recursion is. From what I remember, recursion is when a function calls itself to solve smaller instances of the same problem. So, like, instead of looping through a list to find something, the function talks to itself over and over. But why would business programming use that? Maybe for things that can be broken down into smaller, similar tasks?

Wait, business applications are usually about processing data, managing orders, handling transactions, etc. So maybe recursion can help in problems where you need to process each item in a list repeatedly but not in a simple way. Like processing a tree structure? For example, company directories where each department has sub-departments. To get all departments under a certain class, you'd loop through each department and then each sub-department. That's a common use case.

But isn't that too straightforward? Maybe in more complex scenarios, recu

# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](https://github.com/panaversity/learn-agentic-ai/blob/main/01_ai_agents_first/02_openrouter/openrouter.png?raw=1)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```